In [1]:
import wabi
import sys

In [ ]:
sys.path

In [2]:
 from wabi import graphql2pay

In [3]:
import json
import requests as req
import pandas as pd

In [4]:
from wabi import sql_wabilytics, maps_2b

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__init__.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps\__

C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wabi\maps_2b\__init__.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  zonas=zonas.append(ciudad.zonas,ignore_index=True)
C:\Users\camilafernandez_wabi\AppData\Local\Programs\Python\Python310\lib\site-packages\wab

In [1]:
suppliers_wallet = """
SELECT
	s.wabipay_username 
FROM wabi2b.suppliers as s
WHERE s.wabipay_username IS NOT NULL
LIMIT 4;
"""

In [7]:
suppliers_df = sql_wabilytics.querySQL(suppliers_wallet,'wabi2b')

In [8]:
suppliers_df

,wabipay_username
0,pitcofoods
1,Irish_Store
2,Omaria Market
3,NunoACwabi


In [9]:
for i in range(0,len(suppliers_df)):
    suppliers_df.loc[i,'Type'] = 'Supplier'

In [10]:
suppliers_df

,wabipay_username,Type
0,pitcofoods,Supplier
1,Irish_Store,Supplier
2,Omaria Market,Supplier
3,NunoACwabi,Supplier


In [11]:
stores_wallets = """
SELECT 
	s.wabipay_account
FROM wabiRojo.stores s 
WHERE s.wabipay_account IS NOT NULL
LIMIT 2;
"""

In [12]:
stores_df = sql_wabilytics.querySQL(stores_wallets,'wabiRojo')

In [13]:
stores_df

,wabipay_account
0,ws.padua.m
1,ws.ruasaojorge


In [14]:
for i in range(0,len(stores_df)):
    stores_df.loc[i,'Type'] = 'Store'

In [15]:
stores_df = stores_df.rename(columns = {"wabipay_account":"wabipay_username"})
stores_df

,wabipay_username,Type
0,ws.padua.m,Store
1,ws.ruasaojorge,Store


In [16]:
restaurants_wallets = """
SELECT
	rw.wabipay_username
FROM wabiRojo.restaurant_wallet rw 
WHERE rw.wabipay_username IS NOT NULL 
LIMIT 3;
"""

In [17]:
restaurants_df = sql_wabilytics.querySQL(restaurants_wallets,'wabiRojo')

In [18]:
restaurants_df

,wabipay_username
0,xing_tang
1,saleotra.bebidas
2,tijuanapizzaparty


In [19]:
for i in range(0,len(restaurants_df)):
    restaurants_df.loc[i,'Type'] = 'Restaurant'

In [20]:
restaurants_df

,wabipay_username,Type
0,xing_tang,Restaurant
1,saleotra.bebidas,Restaurant
2,tijuanapizzaparty,Restaurant


In [21]:
final_df = pd.concat([suppliers_df,stores_df,restaurants_df]).reset_index(drop=True)

In [22]:
final_df.loc[len(final_df),'wabipay_username'] = 'Wabi2B'
#agregar la wallet de wabi2B

In [23]:
final_df

,wabipay_username,Type
0,pitcofoods,Supplier
1,Irish_Store,Supplier
2,Omaria Market,Supplier
3,NunoACwabi,Supplier
4,ws.padua.m,Store
5,ws.ruasaojorge,Store
6,xing_tang,Restaurant
7,saleotra.bebidas,Restaurant
8,tijuanapizzaparty,Restaurant
9,Wabi2B,NaN


In [26]:
graphql2pay.bypassToken()
access_token = graphql2pay.token

Got it!


In [28]:
def query_maker(i,token):
    
    query = '''
    query{
  findUserByUsername(input:{
    username:"'''+final_df.loc[i,'wabipay_username']+'''"
    accessToken:"'''+token+'''"
  }){
    ... on User{
      accountBalances{
        ... on RedeemableAccountBalance{
          
          availableMoney{amount,currency}
          redeemerId
          moneyCategory
          redeemerType
          
          
        }
        ... on WithdrawableAccountBalance{
          availableMoney{amount,currency}
          moneyCategory
        }
        
      }
    }
  }
}

    '''
    return query

In [61]:
#RedeemableAccountBalance = plata "real" en la cuenta
#WithdrawableAccountBalance = plata en puntos en la cuenta
results = [] #esto será una lista de diccionarios
url = 'https://api.prod.yopago.app/graphql'
for i in range(0,len(final_df)):
    query = query_maker(i,access_token)
    response = req.post(url = url, json = {"query":query})
    results.append(json.loads(response.content))

In [62]:
results

[{'extensions': {'duration': 15712704},
  'data': {'findUserByUsername': {'accountBalances': []}}},
 {'extensions': {'duration': 21181632},
  'data': {'findUserByUsername': {'accountBalances': [{'availableMoney': {'amount': 0.0,
       'currency': 'PHP'},
      'redeemerId': 'eb966cd4-a172-4d7d-9b28-f700653ef0a6',
      'moneyCategory': 'REDEEMABLE',
      'redeemerType': 'LOYALTY_PROGRAM'}]}}},
 {'extensions': {'duration': 6810524}, 'data': {'findUserByUsername': {}}},
 {'extensions': {'duration': 20127850},
  'data': {'findUserByUsername': {'accountBalances': [{'availableMoney': {'amount': 306.15,
       'currency': 'EUR'},
      'redeemerId': 'eb966cd4-a172-4d7d-9b28-f700653ef0a6',
      'moneyCategory': 'REDEEMABLE',
      'redeemerType': 'LOYALTY_PROGRAM'},
     {'availableMoney': {'amount': 0.0, 'currency': 'EUR'},
      'moneyCategory': 'WITHDRAWABLE'}]}}},
 {'extensions': {'duration': 88388121},
  'data': {'findUserByUsername': {'accountBalances': [{'availableMoney': {'amount':

In [68]:
i = 0
j = 0
results_df = pd.DataFrame()
for json_result in results:
    if json_result['data']['findUserByUsername']:
        #print("acá la lista de diccionarios")
        #print(json_result['data']['findUserByUsername'])
        for dictionary in json_result['data']['findUserByUsername']["accountBalances"]:
            if dictionary['moneyCategory'] == 'REDEEMABLE':
                results_df.loc[j,'wabipay_username'] = final_df.loc[i,'wabipay_username']
                results_df.loc[j,'Type'] = final_df.loc[i,'Type']
                results_df.loc[j,'amount'] = dictionary['availableMoney']['amount']
                results_df.loc[j,'currency'] = dictionary['availableMoney']['currency']
                results_df.loc[j,'redeemerId'] = dictionary['redeemerId']
                results_df.loc[j,'moneyCategory'] = dictionary['moneyCategory']
                results_df.loc[j,'redeemerType'] = dictionary['redeemerType']
                j+=1
            else:
                results_df.loc[j,'wabipay_username'] = final_df.loc[i,'wabipay_username']
                results_df.loc[j,'Type'] = final_df.loc[i,'Type']
                results_df.loc[j,'amount'] = dictionary['availableMoney']['amount']
                results_df.loc[j,'currency'] = dictionary['availableMoney']['currency']
                results_df.loc[j,'moneyCategory'] = dictionary['moneyCategory']
                j+=1
            #print(final_df.loc[i,'wabipay_username'])
            #print("acá el diccionario")
            #print(dictionary)
            
        i+=1
            
    else:
        i+=1
        pass
                
        

In [70]:
results_df

,wabipay_username,Type,amount,currency,redeemerId,moneyCategory,redeemerType
0,Irish_Store,Supplier,0.000000e+00,PHP,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM
1,NunoACwabi,Supplier,3.061500e+02,EUR,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM
2,NunoACwabi,Supplier,0.000000e+00,EUR,NaN,WITHDRAWABLE,NaN
3,ws.padua.m,Store,0.000000e+00,ARS,4d626a3e-bb20-4a5d-a90e-5c30a8a75533,REDEEMABLE,LOYALTY_PROGRAM
4,ws.padua.m,Store,6.104310e+03,ARS,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM
5,ws.padua.m,Store,3.141000e+03,ARS,NaN,WITHDRAWABLE,NaN
6,ws.ruasaojorge,Store,0.000000e+00,BRL,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM
7,ws.ruasaojorge,Store,3.850000e+01,BRL,NaN,WITHDRAWABLE,NaN
8,saleotra.bebidas,Restaurant,0.000000e+00,ARS,NaN,WITHDRAWABLE,NaN
9,Wabi2B,NaN,2.993056e+06,ARS,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM


In [72]:
loyalty_program_query = """
SELECT 
	lp.loyalty_program_id,
	lp.loyalty_program_name 
FROM wabipay.loyalty_program lp"""

In [77]:
loyalty_program_df = sql_wabilytics.querySQL(loyalty_program_query,'wabipay')

In [78]:
loyalty_program_df = loyalty_program_df.rename(columns={'loyalty_program_id':'redeemerId'})

In [79]:
loyalty_program_df

,redeemerId,loyalty_program_name
0,08a7c973-dad6-11ec-a007-0232d2d70db3,WabiPuntos
1,123c14ec-cff9-4f99-b8f8-226b415b9dfc,WabiCarepuntos
2,4d626a3e-bb20-4a5d-a90e-5c30a8a75533,LydarPuntos
3,5746370c-1a16-4f7e-85ea-5099f7455ae1,WabiMillas
4,617469d0-1ed9-450f-8a09-b4e1b29f8cc0,WabiLoversPuntos
5,7d9d9f43-8753-4b50-8132-8aa92de2d6aa,WabiPayPoints
6,872c9c5e-d204-4e0f-93f2-cc2c7a1bf715,Coca-ColaPoints
7,a69ba31f-9c22-4040-9890-e44e9cbd8c9a,McdonaldsPuntos
8,aac02faf-58cc-480c-ba5e-c08331578bbf,RabietaPuntos
9,c10f9546-ce59-437a-86c4-c677f8afa9a9,prueba


In [82]:
new_df = results_df.merge(loyalty_program_df,how = 'left', on = 'redeemerId').reset_index(drop = True)

In [84]:
new_df

,wabipay_username,Type,amount,currency,redeemerId,moneyCategory,redeemerType,loyalty_program_name
0,Irish_Store,Supplier,0.000000e+00,PHP,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM,Wabi2Bpuntos
1,NunoACwabi,Supplier,3.061500e+02,EUR,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM,Wabi2Bpuntos
2,NunoACwabi,Supplier,0.000000e+00,EUR,NaN,WITHDRAWABLE,NaN,NaN
3,ws.padua.m,Store,0.000000e+00,ARS,4d626a3e-bb20-4a5d-a90e-5c30a8a75533,REDEEMABLE,LOYALTY_PROGRAM,LydarPuntos
4,ws.padua.m,Store,6.104310e+03,ARS,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM,Wabi2Bpuntos
5,ws.padua.m,Store,3.141000e+03,ARS,NaN,WITHDRAWABLE,NaN,NaN
6,ws.ruasaojorge,Store,0.000000e+00,BRL,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM,Wabi2Bpuntos
7,ws.ruasaojorge,Store,3.850000e+01,BRL,NaN,WITHDRAWABLE,NaN,NaN
8,saleotra.bebidas,Restaurant,0.000000e+00,ARS,NaN,WITHDRAWABLE,NaN,NaN
9,Wabi2B,NaN,2.993056e+06,ARS,eb966cd4-a172-4d7d-9b28-f700653ef0a6,REDEEMABLE,LOYALTY_PROGRAM,Wabi2Bpuntos


In [86]:
new_df = new_df.groupby(['wabipay_username','moneyCategory','loyalty_program_name']).sum('amount').reset_index()

In [88]:
new_df

,wabipay_username,moneyCategory,loyalty_program_name,amount
0,Irish_Store,REDEEMABLE,Wabi2Bpuntos,0.000000e+00
1,NunoACwabi,REDEEMABLE,Wabi2Bpuntos,3.061500e+02
2,Wabi2B,REDEEMABLE,Wabi2Bpuntos,5.414136e+09
3,ws.padua.m,REDEEMABLE,LydarPuntos,0.000000e+00
4,ws.padua.m,REDEEMABLE,Wabi2Bpuntos,6.104310e+03
5,ws.ruasaojorge,REDEEMABLE,Wabi2Bpuntos,0.000000e+00
